In [2]:
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 --extra-index-url https://download.pytorch.org/whl/cu118
!import os
import glob
import numpy as np
import netCDF4 as nc
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import GradScaler, autocast  # Cambiado de torch.amp a torch.cuda.amp
import matplotlib.pyplot as pltpip install netCDF4==1.6.5 numpy==1.24.3 matplotlib==3.7.1 jupyter==1.0.0 scipy==1.10.1 h5py==3.9.0 tqdm==4.66.1

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 2.6 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 65.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 43.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 41.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 65.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 54.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 20.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 25.1
[notice] To update, run: python3 -m pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━

In [8]:
import torch
import netCDF4, numpy, matplotlib, scipy, h5py, tqdm
from torch.cuda.amp import GradScaler, autocast
print(torch.__version__, torch.cuda.is_available(), torch.cuda.get_device_name(0))
print("Dependencias OK")

2.0.1+cu118 True NVIDIA A100-PCIE-40GB
Dependencias OK


In [9]:
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 --extra-index-url https://download.pytorch.org/whl/cu118
!pip install netCDF4==1.6.5 numpy==1.24.3 matplotlib==3.7.1 jupyter==1.0.0 scipy==1.10.1 h5py==3.10.0 tqdm==4.66.1

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118

[notice] A new release of pip is available: 24.0 -> 25.1
[notice] To update, run: python3 -m pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 11.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0

[notice] A new release of pip is available: 24.0 -> 25.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [10]:
!mkdir -p /home/model_output
!chmod -R 777 /home/model_output
!ln -sf /home/f-caballero/UM/netCDF_Big_sample_new_range /home/first_try_nc

In [13]:
!ls /home/first_try_nc


2020020320  2020020326	2020020332  2020020338
2020020322  2020020328	2020020334  netCDF_Big_sample_new_range
2020020324  2020020330	2020020336


In [14]:
!find /home/first_try_nc -name "*.nc"

/home/first_try_nc/2020020320/042405.nc
/home/first_try_nc/2020020320/040919.nc
/home/first_try_nc/2020020320/041812.nc
/home/first_try_nc/2020020320/041514.nc
/home/first_try_nc/2020020320/042704.nc
/home/first_try_nc/2020020320/042109.nc
/home/first_try_nc/2020020320/041216.nc
/home/first_try_nc/2020020326/171506.nc
/home/first_try_nc/2020020326/172354.nc
/home/first_try_nc/2020020326/172058.nc
/home/first_try_nc/2020020326/170614.nc
/home/first_try_nc/2020020326/171209.nc
/home/first_try_nc/2020020326/171803.nc
/home/first_try_nc/2020020326/170911.nc
/home/first_try_nc/2020020330/184933.nc
/home/first_try_nc/2020020330/190419.nc
/home/first_try_nc/2020020330/185826.nc
/home/first_try_nc/2020020330/185230.nc
/home/first_try_nc/2020020330/190124.nc
/home/first_try_nc/2020020330/190715.nc
/home/first_try_nc/2020020330/185528.nc
/home/first_try_nc/2020020328/180209.nc
/home/first_try_nc/2020020328/175912.nc
/home/first_try_nc/2020020328/175317.nc
/home/first_try_nc/2020020328/174724.nc


In [16]:
!chmod -R 777 /home/first_try_nc
!chmod -R 777 /home/model_output

In [18]:
import netCDF4
data = netCDF4.Dataset('/home/first_try_nc/2020020320/040919.nc')
print(data.variables['DBZ'].shape)
# Esperado: (1, 18, 500, 500)

(1, 18, 500, 500)


In [25]:
import os
import glob as glob_func
import numpy as np
import netCDF4 as nc
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import GradScaler, autocast
import matplotlib.pyplot as plt

# Configuración
data_dir = "/home/first_try_nc"
output_dir = "/home/model_output"
seq_len = 6  # Pasos de entrada
pred_len = 1  # Pasos de salida
batch_size = 1
epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resolution = (18, 500, 500)  # z, y, x

# Liberar memoria de la GPU
torch.cuda.empty_cache()

# Implementación personalizada de ConvLSTM2D
class ConvLSTMCell(nn.Module):
    def __init__(self, input_dim, hidden_dim, kernel_size, bias=True):
        super(ConvLSTMCell, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.padding = kernel_size // 2
        self.bias = bias

        self.conv = nn.Conv3d(
            in_channels=input_dim + hidden_dim,
            out_channels=4 * hidden_dim,
            kernel_size=(kernel_size, kernel_size, kernel_size),
            padding=(self.padding, self.padding, self.padding),
            bias=bias
        )

    def forward(self, input_tensor, cur_state):
        h_cur, c_cur = cur_state
        combined = torch.cat([input_tensor, h_cur], dim=1)
        combined_conv = self.conv(combined)
        cc_i, cc_f, cc_c, cc_o = torch.split(combined_conv, self.hidden_dim, dim=1)
        i = torch.sigmoid(cc_i)
        f = torch.sigmoid(cc_f)
        c_next = f * c_cur + i * torch.tanh(cc_c)
        o = torch.sigmoid(cc_o)
        h_next = o * torch.tanh(c_next)
        return h_next, c_next

    def init_hidden(self, batch_size, image_size):
        z, height, width = image_size
        return (torch.zeros(batch_size, self.hidden_dim, z, height, width, device=self.conv.weight.device),
                torch.zeros(batch_size, self.hidden_dim, z, height, width, device=self.conv.weight.device))

class ConvLSTM2D(nn.Module):
    def __init__(self, input_dim, hidden_dim, kernel_size, num_layers, batch_first=False, bias=True):
        super(ConvLSTM2D, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.num_layers = num_layers
        self.batch_first = batch_first

        self.cells = nn.ModuleList([
            ConvLSTMCell(
                input_dim if i == 0 else hidden_dim,
                hidden_dim,
                kernel_size,
                bias
            ) for i in range(num_layers)
        ])

    def forward(self, input_tensor, hidden_state=None):
        if self.batch_first:
            input_tensor = input_tensor.permute(0, 1, 5, 2, 3, 4)  # (batch, seq_len, channels, z, height, width)
        else:
            input_tensor = input_tensor.permute(1, 0, 5, 2, 3, 4)

        batch_size, seq_len, channels, z, height, width = input_tensor.size()
        if hidden_state is None:
            hidden_state = [
                self.cells[l].init_hidden(batch_size, (z, height, width))
                for l in range(self.num_layers)
            ]

        output_inner = []
        for t in range(seq_len):
            cur_input = input_tensor[:, t]  # (batch, channels, z, height, width)
            for layer_idx in range(self.num_layers):
                h, c = hidden_state[layer_idx]
                h, c = self.cells[layer_idx](cur_input, (h, c))
                hidden_state[layer_idx] = (h, c)
                cur_input = h  # Pasar salida como entrada a la siguiente capa
            output_inner.append(h)

        output = torch.stack(output_inner, dim=1 if self.batch_first else 0)
        return output, hidden_state

# Dataset
class RadarDataset(Dataset):
    def __init__(self, data_dir, seq_len=6, pred_len=1, min_dbz=-30, max_dbz=70):
        self.seq_len = seq_len
        self.pred_len = pred_len
        self.min_dbz = min_dbz
        self.max_dbz = max_dbz
        self.folders = sorted([f for f in glob_func.glob(os.path.join(data_dir, "*")) if os.path.isdir(f)])
        if not self.folders:
            raise ValueError(f"No se encontraron subcarpetas en {data_dir}")
        
        self.sequences = []
        for folder in self.folders:
            files = sorted(glob_func.glob(os.path.join(folder, "*.nc")))
            if len(files) >= seq_len + pred_len:
                for i in range(len(files) - seq_len - pred_len + 1):
                    self.sequences.append(files[i:i + seq_len + pred_len])
        if not self.sequences:
            raise ValueError(f"No se encontraron secuencias válidas")

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        seq_files = self.sequences[idx]
        x = []
        for f in seq_files[:self.seq_len]:
            ds = nc.Dataset(f)
            dbz = ds.variables['DBZ'][:]  # (1, 18, 500, 500)
            ds.close()
            dbz = (dbz - self.min_dbz) / (self.max_dbz - self.min_dbz)  # Normaliza [-30, 70] a [0, 1]
            x.append(dbz[0])
        x = np.stack(x, axis=0)  # (6, 18, 500, 500)
        y = []
        for f in seq_files[self.seq_len:self.seq_len + pred_len]:
            ds = nc.Dataset(f)
            dbz = ds.variables['DBZ'][:]  # (1, 18, 500, 500)
            ds.close()
            dbz = (dbz - self.min_dbz) / (self.max_dbz - self.min_dbz)  # Normaliza [-30, 70] a [0, 1]
            y.append(dbz[0])
        y = np.stack(y, axis=0)  # (1, 18, 500, 500)
        x_tensor = torch.tensor(x, dtype=torch.float32).unsqueeze(-1)  # (6, 18, 500, 500, 1)
        y_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(-1)  # (1, 18, 500, 500, 1)
        return x_tensor, y_tensor

# Modelo ConvLSTM
class ConvLSTM(nn.Module):
    def __init__(self, in_channels=1, hidden_channels=8, num_layers=2, kernel_size=3):
        super(ConvLSTM, self).__init__()
        self.convlstm = ConvLSTM2D(
            input_dim=in_channels,
            hidden_dim=hidden_channels,
            kernel_size=kernel_size,
            num_layers=num_layers,
            batch_first=True
        )
        self.conv = nn.Conv3d(hidden_channels, 1, kernel_size=(kernel_size, kernel_size, kernel_size),
                              padding=(1, 1, 1))

    def forward(self, x):
        x, _ = self.convlstm(x)
        x = x[:, -1]
        x = self.conv(x)
        return x

# Crear directorio de salida
os.makedirs(output_dir, exist_ok=True)

# Cargar datos
try:
    dataset = RadarDataset(data_dir, seq_len, pred_len, min_dbz=-30, max_dbz=70)
except ValueError as e:
    print(f"Error en dataset: {e}")
    exit(1)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

# Inicializar modelo
model = ConvLSTM(hidden_channels=8, num_layers=2).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()
scaler = GradScaler('cuda')

# Entrenamiento
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for x, y in dataloader:
        print("Forma de x:", x.shape)  # Depuración: verificar forma de x
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        with autocast():
            y_pred = model(x)
            loss = criterion(y_pred, y[:, 0].squeeze(-1))
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(dataloader):.6f}")

# Guardar modelo
torch.save(model.state_dict(), os.path.join(output_dir, "convlstm.pth"))

# Generar predicciones (ejemplo)
model.eval()
with torch.no_grad():
    x, y = dataset[0]
    x = x.unsqueeze(0).to(device)  # (1, 6, 18, 500, 500, 1)
    y_pred = model(x)  # (1, 1, 18, 500, 500)
    y_pred = y_pred.cpu().numpy() * (70 - (-30)) + (-30)  # Escala [0, 1] a [-30, 70]
    y = y.numpy() * (70 - (-30)) + (-30)  # Escala [0, 1] a [-30, 70]
    ds_out = nc.Dataset(os.path.join(output_dir, "pred_0.nc"), 'w', format='NETCDF4')
    ds_out.createDimension('time', 1)
    ds_out.createDimension('z', 18)
    ds_out.createDimension('y', 500)
    ds_out.createDimension('x', 500)
    dbz_var = ds_out.createVariable('DBZ', 'f4', ('time', 'z', 'y', 'x'))
    dbz_var[:] = y_pred
    ds_out.close()

Forma de x: torch.Size([1, 6, 18, 500, 500, 1])


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 18, 500, 500, 1])) that is different to the input size (torch.Size([1, 1, 18, 500, 500])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (18) must match the size of tensor b (500) at non-singleton dimension 2

In [21]:
from glob import glob
import os
data_dir = "/home/first_try_nc"
seq_len = 6
pred_len = 1
folders = sorted([f for f in glob(os.path.join(data_dir, "*")) if os.path.isdir(f)])
for folder in folders:
    files = sorted(glob(os.path.join(folder, "*.nc")))
    print(f"Carpeta {folder}: {len(files)} archivos")
    if len(files) >= seq_len + pred_len:
        sequences = [files[i:i + seq_len + pred_len] for i in range(len(files) - seq_len - pred_len + 1)]
        print(f"  Secuencias válidas: {len(sequences)}")
    else:
        print("  No hay suficientes archivos para formar secuencias")

Carpeta /home/first_try_nc/2020020320: 7 archivos
  Secuencias válidas: 1
Carpeta /home/first_try_nc/2020020322: 7 archivos
  Secuencias válidas: 1
Carpeta /home/first_try_nc/2020020324: 7 archivos
  Secuencias válidas: 1
Carpeta /home/first_try_nc/2020020326: 7 archivos
  Secuencias válidas: 1
Carpeta /home/first_try_nc/2020020328: 7 archivos
  Secuencias válidas: 1
Carpeta /home/first_try_nc/2020020330: 7 archivos
  Secuencias válidas: 1
Carpeta /home/first_try_nc/2020020332: 7 archivos
  Secuencias válidas: 1
Carpeta /home/first_try_nc/2020020334: 7 archivos
  Secuencias válidas: 1
Carpeta /home/first_try_nc/2020020336: 7 archivos
  Secuencias válidas: 1
Carpeta /home/first_try_nc/2020020338: 7 archivos
  Secuencias válidas: 1
